In [16]:
import pandas as pd

# Replace 'your_file.parquet' with your file path
df = pd.read_parquet('test_lang.parquet')

# Preview the data
print(df.head(10))
print(df.columns.tolist())
# Split training data
with open('test_en', 'w', encoding='utf-8') as f_en, \
     open('test_hi', 'w', encoding='utf-8') as f_hi:
    for _, row in df.iterrows():
        f_en.write(row['translation']['en'] + '\n')
        f_hi.write(row['translation']['hi'] + '\n')

# Replace 'your_file.parquet' with your file path
df = pd.read_parquet('train_lang.parquet')

# Preview the data
print(df.head(10))
print(df.columns.tolist())
# Split training data
with open('train_en', 'w', encoding='utf-8') as f_en, \
     open('train_hi', 'w', encoding='utf-8') as f_hi:
    for _, row in df.iterrows():
        f_en.write(row['translation']['en'] + '\n')
        f_hi.write(row['translation']['hi'] + '\n')

# Replace 'your_file.parquet' with your file path
df = pd.read_parquet('val_lang.parquet')

# Preview the data
print(df.head(10))
print(df.columns.tolist())
# Split training data
with open('val_en', 'w', encoding='utf-8') as f_en, \
     open('val_hi', 'w', encoding='utf-8') as f_hi:
    for _, row in df.iterrows():
        f_en.write(row['translation']['en'] + '\n')
        f_hi.write(row['translation']['hi'] + '\n')

                                         translation
0  {'en': 'A black box in your car?', 'hi': 'आपकी...
1  {'en': 'As America's road planners struggle to...
2  {'en': 'The devices, which track every mile a ...
3  {'en': 'The usually dull arena of highway plan...
4  {'en': 'Libertarians have joined environmental...
5  {'en': 'The tea party is aghast.', 'hi': 'चाय ...
6  {'en': 'The American Civil Liberties Union is ...
7  {'en': 'And while Congress can't agree on whet...
8  {'en': 'They are exploring how, over the next ...
9  {'en': 'Thousands of motorists have already ta...
['translation']
                                         translation
0  {'en': 'Give your application an accessibility...
1  {'en': 'Accerciser Accessibility Explorer', 'h...
2  {'en': 'The default plugin layout for the bott...
3  {'en': 'The default plugin layout for the top ...
4  {'en': 'A list of plugins that are disabled by...
5  {'en': 'Highlight duration', 'hi': 'अवधि को हा...
6  {'en': 'The duration of the

In [17]:
from tokenizers import ByteLevelBPETokenizer
en_tokenizer = ByteLevelBPETokenizer()
en_tokenizer.train(
    files=['train_en', 'test_en', 'val_en'],
    vocab_size=10000,
    min_frequency=2,
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]']
)
en_tokenizer.save_model('.', 'english_bpe')

# 4. Train Hindi BPE Tokenizer (handles Devanagari script)
hi_tokenizer = ByteLevelBPETokenizer()
hi_tokenizer.train(
    files=['train_hi', 'test_hi', 'val_hi'],
    vocab_size=15000,  # Larger vocab for Hindi's richer morphology
    min_frequency=2,
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]']
)
hi_tokenizer.save_model('.', 'hindi_bpe')


['./hindi_bpe-vocab.json', './hindi_bpe-merges.txt']

In [1]:
from tokenizers import ByteLevelBPETokenizer

In [2]:
# Load saved tokenizers
en_tokenizer = ByteLevelBPETokenizer(
    'english_bpe-vocab.json',
    'english_bpe-merges.txt'
)
hi_tokenizer = ByteLevelBPETokenizer(
    'hindi_bpe-vocab.json',
    'hindi_bpe-merges.txt'
)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
# Custom Multi-Head Attention
class CustomMultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.15):
        super().__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.dropout = dropout
        self.head_dim = d_model // nhead

        assert d_model % nhead == 0,  "d_model must be divisible by nhead"

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        B, T_q, _ = query.size()
        B, T_k, _ = key.size()
        B, T_v, _ = value.size()

        Q = self.q_linear(query).view(B, T_q, self.nhead, self.head_dim).transpose(1, 2)
        K = self.k_linear(key).view(B, T_k, self.nhead, self.head_dim).transpose(1, 2)
        V = self.v_linear(value).view(B, T_v, self.nhead, self.head_dim).transpose(1, 2)
        #print(f"Q shape: {Q.shape}, K shape: {K.shape}, V shape: {V.shape}")

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            #mask = mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(mask == 0, -float('inf'))

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout_layer(attn)
        #print(f"Attn Shape: {attn.shape}")
        output = torch.matmul(attn, V)
        #print(f"output.shape: {output.shape}")
        #print(f"B: {B}, T_Q: {T_q}, T_K: {T_k}, T_V: {T_v}, d_model: {self.d_model}")

        output = output.transpose(1, 2).contiguous().view(B, T_q, self.d_model)
        return self.out_linear(output)

In [5]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))


In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = CustomMultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        src2 = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout(src2))
        src2 = self.ffn(src)
        src = self.norm2(src + self.dropout(src2))
        return src

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = CustomMultiHeadAttention(d_model, num_heads)
        self.cross_attn = CustomMultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask, memory_mask):
        tgt2 = self.self_attn(tgt, tgt, tgt, tgt_mask)
        tgt = self.norm1(tgt + self.dropout(tgt2))
        tgt2 = self.cross_attn(tgt, memory, memory, memory_mask)
        tgt = self.norm2(tgt + self.dropout(tgt2))
        tgt2 = self.ffn(tgt)
        tgt = self.norm3(tgt + self.dropout(tgt2))
        return tgt


In [7]:
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1)]
        return x



In [8]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout=0.1, max_len=512):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, src, src_mask):
        x = self.embed(src)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, src_mask)
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout=0.1, max_len=512):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, tgt, memory, tgt_mask, memory_mask):
        x = self.embed(tgt)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, memory, tgt_mask, memory_mask)
        return self.fc_out(x)


In [9]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=256, num_layers=4, num_heads=4, d_ff=1024, dropout=0.1, max_len=512):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_len)

    def make_src_mask(self, src, pad_token_id):
        return (src != pad_token_id).unsqueeze(1).unsqueeze(2)

    def make_tgt_mask(self, tgt, pad_token_id):
        batch_size, tgt_len = tgt.shape
        tgt_mask = (tgt != pad_token_id).unsqueeze(1).unsqueeze(2)
        nopeak_mask = torch.tril(torch.ones((tgt_len, tgt_len), device=tgt.device)).bool().unsqueeze(0).unsqueeze(0)
        return tgt_mask & nopeak_mask

    def forward(self, src, tgt_input, src_mask, tgt_mask):
        memory = self.encoder(src, src_mask)
        output = self.decoder(tgt_input, memory, tgt_mask, src_mask)
        return output



In [ ]:
# Training Params
batch_size = 64        
num_epochs = 25        # for testing
learning_rate = 0.01 # Adam optimizer
dropout = 0.15          # Regularization


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import random
class TranslationDataset(Dataset):
    def __init__(self, parquet_path, en_tokenizer, hi_tokenizer, max_length=128):
        self.df = pd.read_parquet(parquet_path)
        self.en_tok = en_tokenizer
        self.hi_tok = hi_tokenizer
        self.max_len = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        translation = self.df.iloc[idx]['translation']
        en_text = translation['en']
        hi_text = translation['hi']
        
        # Tokenize with special tokens
        en_ids = [self.en_tok.token_to_id('[SOS]')] \
                + self.en_tok.encode(en_text).ids[:self.max_len-2] \
                + [self.en_tok.token_to_id('[EOS]')]
        
        hi_ids = [self.hi_tok.token_to_id('[SOS]')] \
                + self.hi_tok.encode(hi_text).ids[:self.max_len-2] \
                + [self.hi_tok.token_to_id('[EOS]')]

        # Pad sequences
        en_padded = en_ids + [self.en_tok.token_to_id('[PAD]')]*(self.max_len - len(en_ids))
        hi_padded = hi_ids + [self.hi_tok.token_to_id('[PAD]')]*(self.max_len - len(hi_ids))
        
        return {
            'en': torch.tensor(en_padded, dtype=torch.long),
            'hi': torch.tensor(hi_padded, dtype=torch.long)
        }

# Full train dataset
full_train_dataset = TranslationDataset('train_lang.parquet', en_tokenizer, hi_tokenizer)

# select only some dataset to check working
total_len = len(full_train_dataset)
sample_size = int(100)
subset_train_indices = random.sample(range(total_len), sample_size)

# Create subset dataset
train_dataset = Subset(full_train_dataset, subset_train_indices)
full_val_dataset = TranslationDataset('val_lang.parquet', en_tokenizer, hi_tokenizer)

val_len = len(full_val_dataset)
subset_val_indices = random.sample(range(val_len), sample_size)
val_dataset = Subset(full_val_dataset, subset_val_indices)
def collate_fn(batch):
    # Convert list of dicts to tensor tuples
    en = torch.stack([item['en'] for item in batch])
    hi = torch.stack([item['hi'] for item in batch])
    return en, hi

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn  
)

val_loader = DataLoader(val_dataset, batch_size=32,collate_fn=collate_fn)


In [ ]:
model = Transformer(src_vocab_size=10000, tgt_vocab_size=15000)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Training loop
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for src, tgt in train_loader:
        src, tgt = src.to(device), tgt.to(device)
        
        # Teacher forcing preparation
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        
        # Mask creation
        src_mask = model.make_src_mask(src, en_tokenizer.token_to_id("[PAD]"))
        tgt_mask = model.make_tgt_mask(tgt_input, hi_tokenizer.token_to_id("[PAD]"))
        
        # Forward pass
        logits = model(src, tgt_input, src_mask, tgt_mask)
        
        # Loss 
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            tgt_output.reshape(-1),
            ignore_index=hi_tokenizer.token_to_id("[PAD]")
        )
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()

        # Accuracy
        preds = logits.argmax(dim=-1)
        mask = tgt_output != hi_tokenizer.token_to_id("[PAD]")
        train_correct += (preds[mask] == tgt_output[mask]).sum().item()
        train_total += mask.sum().item()

    train_loss /= len(train_loader)
    train_acc = train_correct / train_total

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            src_mask = model.make_src_mask(src, en_tokenizer.token_to_id("[PAD]"))
            tgt_mask = model.make_tgt_mask(tgt_input, hi_tokenizer.token_to_id("[PAD]"))

            logits = model(src, tgt_input, src_mask, tgt_mask)

            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                tgt_output.reshape(-1),
                ignore_index=hi_tokenizer.token_to_id("[PAD]")
            )

            val_loss += loss.item()

            preds = logits.argmax(dim=-1)
            mask = tgt_output != hi_tokenizer.token_to_id("[PAD]")
            val_correct += (preds[mask] == tgt_output[mask]).sum().item()
            val_total += mask.sum().item()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("✅ Best model saved.")


Epoch 1/25 | Train Loss: 7.3572, Train Acc: 5.59% | Val Loss: 5.6909, Val Acc: 1.50%
✅ Best model saved.
Epoch 2/25 | Train Loss: 5.1104, Train Acc: 4.84% | Val Loss: 5.7439, Val Acc: 5.91%
Epoch 3/25 | Train Loss: 4.8737, Train Acc: 5.73% | Val Loss: 5.9189, Val Acc: 5.16%
Epoch 4/25 | Train Loss: 4.9572, Train Acc: 6.19% | Val Loss: 5.9260, Val Acc: 9.61%
Epoch 5/25 | Train Loss: 4.9317, Train Acc: 6.50% | Val Loss: 5.9521, Val Acc: 5.16%
Epoch 6/25 | Train Loss: 4.7614, Train Acc: 5.95% | Val Loss: 5.8725, Val Acc: 9.61%
Epoch 7/25 | Train Loss: 4.8107, Train Acc: 8.24% | Val Loss: 5.8276, Val Acc: 5.16%
Epoch 8/25 | Train Loss: 4.7606, Train Acc: 6.31% | Val Loss: 5.7401, Val Acc: 9.61%
Epoch 9/25 | Train Loss: 4.6922, Train Acc: 8.11% | Val Loss: 5.7251, Val Acc: 9.61%
Epoch 10/25 | Train Loss: 4.7596, Train Acc: 7.95% | Val Loss: 5.6601, Val Acc: 9.61%
✅ Best model saved.
Epoch 11/25 | Train Loss: 4.6858, Train Acc: 7.99% | Val Loss: 5.5932, Val Acc: 9.61%
✅ Best model saved.
Epo